# langchain源码解读

## langchain模块

- langchain
- langserve
- langchain-js
- langsmith
- langfuse

## LCEL的内部实现

### 类结构派生层次

- Runnable
    - RunnableSerializable（**LCEL**规则主要由**RunnableSequence**和**RunnableParallel**实现）

        **配置能力:**
        - RunnableBindingBase
            - RunnableBinding（向Runnable实例传递参数）
        - DynamicRunnable
            - RunnableConfigurableFields
            - RunnableConfigurableAlternatives

        **流程控制:**
        - RunnablePassthrough（传递额外输入）
        - RunnableSequence（实现顺序执行，可以用|符号或RunnableSequence来构造）
        - RunnableParallel（实现并行执行，可以用Dict或RunnableParallel类来构造，别名RunnableMap）

        **大模型:**
        - BaseLanguageModel
            - BaseLLM（派生其他大模型）
                - LLM（派生其他大模型）
                - OpenAIChat
                - Tongyi

        **提示语:**
        - BasePromptTemplate `[Dict, PromptValue]`
            - StringPromptTemplate（字符串模板）
            - BaseChatPromptTemplate（对话模板）
            - ImagePromptTemplate
            - PipelinePromptTemplate

        **检索器:**
        - BaseRetriever `[RetrieverInput, RetrieverOutput]`（派生各类检索器）

        **Tool:**
        - BaseTool `[Union[str, Dict], Any]`（派生各类工具）

        **输出解析:**      
        - BaseGenerationOutputParser `[Union[str, BaseMessage], T]`
        - BaseOutputParser（派生各类输出解析）


        **输入赋值:**      
        - RunnablePassthrough
        - RunnableAssign
        - RunnablePick

        **已实现Chains:**      
        - Chain（派生各类预制chain）

    **定制能力:**
    - RunnableGenerator（常用于处理输出可能是迭代器结果的chain）
    - RunnableLambda（常用于包装普通函数，别名函数@chain）

## Runnable（提供原语的基类）

**LangChain Runnable** 是一种工作单元, 支持同步、异步、批处理和流式操作。

**LangChain表达式语言(LCEL)** 提供了一种声明式的方法来构建使用大语言模型的生产级程序。

使用 **LCEL** 和 **LangChain Runnable** 构建的程序内在地支持同步、异步、批处理和流式操作。

支持异步可以让托管基于 **LCEL** 程序的服务器更好地扩展,以处理更高的并发负载。

流式输出中间结果的能力可以创建更响应的用户体验。

**Runnable** 模块包含了 **LangChain Runnable** 原语的模式和实现。

### 最主要的原语方法

主要方法:

- invoke/ainvoke: 将单个输入转换为输出。  
- batch/abatch: 有效地将多个输入转换为输出。
- stream/astream: 流式输出单个输入生成的结果。
- astream_log: 流式输出输入的输出以及所选的中间结果。

内置优化:  

- 批处理: 默认情况下,批处理使用线程池执行器并行运行invoke()。 
  <br>可以重写以优化批处理。

- 异步: 带有“a”后缀的方法是异步的。默认情况下,它们使用asyncio的线程池执行同步版本。
  <br>可以重写为原生异步。

所有方法都接受一个可选的config参数,可用于配置执行、添加标签和元数据,以进行跟踪、调试等。  <br/>
Runnable通过input_schema属性、output_schema属性和config_schema方法公开其输入、输出和配置的模式信息。

### 链的基本示例

LangChain表达式语言(LCEL)是一种声明式的方式来将Runnable组合成链。
这样构建的任何链都将自动支持同步、异步、批处理和流操作。
主要的组合原语是RunnableSequence和RunnableParallel。

RunnableSequence按顺序调用一系列runnable,一个runnable的输出作为下一个的输入。
可以使用 | 运算符构造,或者通过向RunnableSequence传入runnable列表。
RunnableParallel并发调用runnable,向每个都提供相同的输入。
可以在序列中使用字典字面量构造,或者通过向RunnableParallel传入字典来构造。

代码示例：

In [251]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, chain

@chain
def a(x):
    return(x + 1)

@chain
def b(x):
    return(x + 2)

@chain
def c(dict):
    return(dict["x"] + dict["y"])
    
sequence = a | b

print(sequence.invoke(1))
print(sequence.batch([1, 2, 3]))

parral = {"x": a, "y": b} | c
print(parral.invoke(1)) # (1+1) + (1+2) = 5

4
[4, 5, 6]
5


### LCEL提供的好处

LangChain Expression Language（LCEL）是一种声明式语言，可轻松组合不同的调用顺序构成 Chain。LCEL 自创立之初就被设计为能够支持将原型投入生产环境，**无需代码更改**，从最简单的“提示+LLM”链到最复杂的链（已有用户成功在生产环境中运行包含数百个步骤的 LCEL Chain）。

LCEL的一些亮点包括：

1. **流支持**：使用 LCEL 构建 Chain 时，你可以获得最佳的首个令牌时间（即从输出开始到首批输出生成的时间）。
对于某些 Chain，这意味着可以直接从LLM流式传输令牌到流输出解析器，从而以与 LLM 提供商输出原始令牌相同的速率获得解析后的、增量的输出。

2. **异步支持**：任何使用 LCEL 构建的链条都可以通过同步API（例如，在 Jupyter 笔记本中进行原型设计时）和异步 API（例如，在 LangServe 服务器中）调用。
这使得相同的代码可用于原型设计和生产环境，具有出色的性能，并能够在同一服务器中处理多个并发请求。

3. **优化的并行执行**：当你的 LCEL 链条有可以并行执行的步骤时（例如，从多个检索器中获取文档），我们会自动执行，无论是在同步还是异步接口中，以实现最小的延迟。

4. **重试和回退**：为 LCEL 链的任何部分配置重试和回退。这是使链在规模上更可靠的绝佳方式。
目前我们正在添加重试/回退的流媒体支持，因此你可以在不增加任何延迟成本的情况下获得增加的可靠性。

5. **访问中间结果**：对于更复杂的链条，访问在最终输出产生之前的中间步骤的结果通常非常有用。
这可以用于让最终用户知道正在发生一些事情，甚至仅用于调试链条。你可以流式传输中间结果，并且在每个LangServe服务器上都可用。

6. **输入和输出模式**：输入和输出模式为每个 LCEL 链提供了从链的结构推断出的 Pydantic 和 JSONSchema 模式。
这可以用于输入和输出的验证，是 LangServe 的一个组成部分。

7. **无缝LangSmith跟踪集成**：随着链条变得越来越复杂，理解每一步发生了什么变得越来越重要。
通过 LCEL，所有步骤都自动记录到 LangSmith，以实现最大的可观察性和可调试性。

8. **无缝LangServe部署集成**：任何使用 LCEL 创建的链都可以轻松地使用 LangServe 进行部署。

原文：[https://python.langchain.com/docs/expression_language]()

#### 流支持

#### 异步支持

#### 优化的并行执行

#### 重试和回退

#### 访问中间结果

#### 输入和输出模式

#### 无缝LangSmith跟踪集成

#### 无缝LangServe部署集成

### 其他标准化方法

所有Runnable都暴露了可用于修改其行为的其他方法(例如,添加重试策略、添加生命周期监听器、使其可配置等)。

这些方法适用于任何Runnable,包括通过组合其他Runnable构造的Runnable链。详细信息请参阅各个方法。

类结构如下：

- RunnableBindingBase
    - RunnableBinding（向Runnable实例传递参数）
    - RunnableWithMessageHistory（支持对话历史）
    - RunnableRetry（支持重试，一般用`.with_retry()`配置）
    - HubRunnable（访问`LangChain Hub`的实例）
    - OpenAIFunctionsRouter
- DynamicRunnable（支持动态配置）
    - RunnableConfigurableFields
- RunnableWithFallbacks（支持报错回滚，一般用`.with_fallbacks()`配置）

其中，有三种方法可以修改配置后并返回新的 **Runnable**，使用时有细微区别：

- `assign`: 直接按字典输出的方式为Runnable修改配置
- `bind`: 绑定任意参数到Runnable，是更为通用的方法
- `with_config`: 绑定一个配置对象，是抽象程度更为高级的方法

另外，还可以配置 **Runnable** 的执行策略：

- `with_listeners`:  绑定监督者
- `with_types`: 继承类型
- `with_retry`: 绑定重试策略
- `with_fallbacks`: 绑定回滚策略

#### with_retry

In [103]:
from langchain_core.runnables import RunnableLambda
import random

def add_one(x: int) -> int:
    return x + 1

def buggy_double(y: int) -> int:
    '''Buggy code that will fail 70% of the time'''
    if random.random() > 0.3:
        print('This code failed, and will probably be retried!')
        raise ValueError('Triggered buggy code')
    return y * 2

chain = (
    RunnableLambda(add_one) |
    RunnableLambda(buggy_double).with_retry( # Retry on failure
        stop_after_attempt = 10,
        wait_exponential_jitter = False
    )
)

print("---RunnableSequence")
print(type(chain.first))
print(type(chain.middle))
print(type(chain.last))
print("---Runnable")
print(chain.get_name())
print(chain.get_prompts())
print(chain.input_schema.schema()) # Show inferred input schema
print(chain.output_schema.schema()) # Show inferred output schema

print(chain.invoke(2)) # invoke the sequence (note the retry above!!)

---RunnableSequence
<class 'langchain_core.runnables.base.RunnableLambda'>
<class 'list'>
<class 'langchain_core.runnables.retry.RunnableRetry'>
---Runnable
RunnableSequence
[]
{'title': 'add_one_input', 'type': 'integer'}
{'title': 'buggy_double_output', 'type': 'integer'}
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
6


### 调试和跟踪

随着链的变长,能够看到中间结果以调试和跟踪链是很有用的。<br>
您可以将全局调试标志设置为True,以为所有链启用调试输出:

In [ ]:
from langchain_core.globals import set_debug
set_debug(True)

chain.invoke(3)

或者,您可以将现有或自定义回调传递给任何给定的链:

In [ ]:
from langchain_core.tracers import ConsoleCallbackHandler
from langchain_core.globals import set_debug
set_debug(False)

chain.invoke(
    3,
    config={'callbacks': [ConsoleCallbackHandler()]} # 这与设置debug为true的效果类似
)

### 绘制LCEL执行结构

In [223]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.graph_draw import draw

def add_one(x: int) -> int:
    return x + 1

chain = (
    RunnableLambda(add_one) | add_one
)

chain.get_graph().print_ascii()

 +---------------+   
 | add_one_input |   
 +---------------+   
          *          
          *          
          *          
+-----------------+  
| Lambda(add_one) |  
+-----------------+  
          *          
          *          
          *          
+-----------------+  
| Lambda(add_one) |  
+-----------------+  
          *          
          *          
          *          
+----------------+   
| add_one_output |   
+----------------+   


## 配置能力

`RunnableBinding` 可以被看作是一个"Runnable对象的装饰器"，它保留了 `Runnable` 的基本特性，即批处理、流处理和异步支持，同时添加了额外的功能。

任何继承自 `Runnable` 的类都可以绑定到一个 `RunnableBinding`。
`Runnable` 提供了一套标准的方法来创建 `RunnableBindings` 或 `RunnableBindings` 的子类（例如，`RunnableRetry`，`RunnableWithFallbacks`）以添加额外的功能。

这些方法包括：
- `bind`：绑定 kwargs 以在运行底层可运行对象时传递。
- `with_config`：绑定配置以在运行底层可运行对象时传递。
- `with_listeners`：将生命周期监听器绑定到底层可运行对象。
- `with_types`：覆盖底层可运行对象的输入和输出类型。
- `with_retry`：将重试策略绑定到底层可运行对象。
- `with_fallbacks`：将回退策略绑定到底层可运行对象。

### 配置能力相关类结构

- Runnable
    - RunnableSerializable
        - RunnableBindingBase
            - RunnableBinding（向Runnable底层传递参数，`.bind()`）
            - RunnableWithMessageHistory（支持对话历史）
            - RunnableRetry（支持重试，`.with_retry()`）
            - HubRunnable（访问`LangChain Hub`的实例）
            - OpenAIFunctionsRouter
        - DynamicRunnable（支持动态配置）
            - RunnableConfigurableFields
            - RunnableConfigurableAlternatives
        - RunnableWithFallbacks（支持报错回滚，`.with_fallbacks()`）
        - RunnableAssign（`.assign()`）
        

### assign

为字典对象或RunnableParallel对象赋值：

In [253]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

### bind

使用`bind`：可以绑定 kwargs 以在运行底层可运行对象时传递。

#### 将 stop 传递给 OpenAI

In [256]:
# 创建一个可运行绑定，它在运行时调用 ChatModel，并传递额外的 kwarg `stop=['-']`。
from langchain_openai import ChatOpenAI
model = ChatOpenAI()
model.invoke('Say "Parrot-MAGIC"', stop=['-']) # 应返回 `Parrot`

# 通过 `bind` 方法（它返回一个新的 RunnableBinding）来简单地使用它
runnable_binding = model.bind(stop=['-'])
runnable_binding.invoke('Say "Parrot-MAGIC"') # 应返回 `Parrot`

AIMessage(content='Parrot')

#### 将 seed 传递给 OpenAI

In [262]:
model.bind(seed=42).invoke("你是什么模型?")

AIMessage(content='我是一个基于神经网络的模型，被称为GPT-3（Generative Pre-trained Transformer 3）。我是由OpenAI开发的，用于自然语言处理和生成文本的任务。')

In [263]:
model.bind(seed=42).invoke("你是什么模型?")

AIMessage(content='我是一个基于神经网络的模型，被称为GPT-3（Generative Pre-trained Transformer 3）。我是由OpenAI开发的，用于自然语言处理和生成文本的任务。')

In [265]:
model.bind(seed=43).invoke("你是什么模型?")

AIMessage(content='我是一个基于人工智能技术的对话系统模型，被称为GPT-3（Generative Pre-trained Transformer 3）。我被训练来理解和生成自然语言，可以提供各种领域的信息和帮助回答问题。')

### 根据配置动态选择Runnable对象

## 用于流程控制的Runnable派生类

Runnable 可以支持包括顺序、分支、条件、迭代等多种控制方式，构建较复杂的有向无环图。

与直接使用python代码控制流程不同的是，下面这些流程控制手段仍然保持以Runnable返回，以便获得LCEL的诸多额外好处，如：Runnable的统一方法、序列化能力、重试能力、回滚能力、Langsmith追踪和Langserve的API集成等。

### 流程控制相关类结构

- Runnable
    - RunnableSerializable
        - RunnableSequence（实现顺序执行，可以用|符号或RunnableSequence来构造）
        - RunnableParallel（实现并行执行，可以用Dict或RunnableParallel类来构造，别名RunnableMap）
        - RunnableEach（实现相同的链迭代多次）
        - RunnableBranch（实现流程分支，根据条件选择不同的链执行）
        - RouterRunnable（实现流程分支，根据枚举值选择不同的链执行）

### RunnableSerializable

主要增加了序列化的方法，可以将Runnable以JSON字符串的方式保存或加载：

- loads / dumps （按JSON字符串）
- load / dumpd （按Dict类型）

#### 将Runnable保存为JSON

In [95]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableGenerator
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from typing import Iterator

model = ChatOpenAI()
chant = (
    ChatPromptTemplate.from_template("Give me a 3 word chant about {topic}")
    | model
    | StrOutputParser()
)

In [96]:
from langchain_core.load import loads, dumps
json = dumps(chant)
print(json)

{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "chat", "ChatPromptTemplate"], "kwargs": {"input_variables": ["topic"], "messages": [{"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "chat", "HumanMessagePromptTemplate"], "kwargs": {"prompt": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["topic"], "template": "Give me a 3 word chant about {topic}", "template_format": "f-string", "partial_variables": {}}}}}]}}, "middle": [{"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}}}], "last": {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "output_parser", "StrOutputParser"], "kwargs": {}}, "name": null}}


#### 从JSON中恢复Runnable并调用

In [99]:
loads(json).invoke({"topic":"苹果"})

'"Apple, sweet success!"'

#### 根据配置动态选择Runnable对象

In [231]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.runnables import ConfigurableField, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Give me a 3 word chant about {topic}")
openai = ChatOpenAI()
llm = OpenAI()
output_parser = StrOutputParser()

configurable_model = llm.configurable_alternatives(
    ConfigurableField(id="model"), 
    default_key="llm_openai", 
    chat_openai=openai
)

configurable_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | configurable_model 
    | output_parser
)

In [192]:
configurable_chain.invoke(
    "ice cream", 
    config={"model": "llm_openai"}
)

'\n\n"Sweet, cold, yum!"'

In [193]:
configurable_chain.invoke(
    "ice cream", 
    config={"model": "chat_openai"}
)

'\n\n"Scoop, swirl, savor!"'

#### 查看有哪些动态选择

### RunnableSequence

RunnableSequence 是最最重要的流程控制组件。

另外，RunnableSequence 也增加了几个属性：
- first: 第一个Runnable组件
- middle: 中间的Runnable组件列表
- last: 最后一个Runnable组件

In [228]:
from langchain_core.runnables import RunnableLambda
import random

def add_one(x: int) -> int:
    return x + 1

def buggy_double(y: int) -> int:
    '''Buggy code that will fail 70% of the time'''
    if random.random() > 0.3:
        print('This code failed, and will probably be retried!')
        raise ValueError('Triggered buggy code')
    return y * 2

chain = (
    RunnableLambda(add_one) |
    RunnableLambda(buggy_double).with_retry( # Retry on failure
        stop_after_attempt = 10,
        wait_exponential_jitter = False
    )
)
chain.get_graph().print_ascii()

print("---RunnableSequence")
print(type(chain.first))
print(type(chain.middle))
print(type(chain.last))

    +---------------+    
    | add_one_input |    
    +---------------+    
            *            
            *            
            *            
  +-----------------+    
  | Lambda(add_one) |    
  +-----------------+    
            *            
            *            
            *            
+----------------------+ 
| Lambda(buggy_double) | 
+----------------------+ 
            *            
            *            
            *            
+---------------------+  
| buggy_double_output |  
+---------------------+  
---RunnableSequence
<class 'langchain_core.runnables.base.RunnableLambda'>
<class 'list'>
<class 'langchain_core.runnables.retry.RunnableRetry'>


### RunnableParallel

一种情况是从同一个输入生成多个分支链：

In [227]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

def mul_three(x: int) -> int:
    return x * 3

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
runnable_3 = RunnableLambda(mul_three)

sequence = runnable_1 | {  # this dict is coerced to a RunnableParallel
    "mul_two": runnable_2,
    "mul_three": runnable_3,
}
# Or equivalently:
# sequence = runnable_1 | RunnableParallel(
#     {"mul_two": runnable_2, "mul_three": runnable_3}
# )
# Also equivalently:
# sequence = runnable_1 | RunnableParallel(
#     mul_two=runnable_2,
#     mul_three=runnable_3,
# )

sequence.invoke(1)
sequence.get_graph().print_ascii()

await sequence.ainvoke(1)

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

                    +---------------+                     
                    | add_one_input |                     
                    +---------------+                     
                            *                             
                            *                             
                            *                             
                  +-----------------+                     
                  | Lambda(add_one) |                     
                  +-----------------+                     
                            *                             
                            *                             
                            *                             
          +----------------------------------+            
          | Parallel<mul_two,mul_three>Input |            
          +----------------------------------+            
                  ***               ***                   
               ***                     ***              

[{'mul_two': 4, 'mul_three': 6},
 {'mul_two': 6, 'mul_three': 9},
 {'mul_two': 8, 'mul_three': 12}]

### RunnableEach

如果需要让同一个链执行多次，使用 RunnableEach 非常方便。

In [224]:
from langchain_core.runnables.base import RunnableEach, chain

@chain
def myfunc(x):
    return(x * 2)

runnable_each = RunnableEach(bound = myfunc)
runnable_each.get_graph().print_ascii()

output = runnable_each.invoke(range(1, 5))
print(output)

 +--------------+  
 | myfunc_input |  
 +--------------+  
         *         
         *         
         *         
+----------------+ 
| Lambda(myfunc) | 
+----------------+ 
         *         
         *         
         *         
+---------------+  
| myfunc_output |  
+---------------+  
[2, 4, 6, 8]


### RunnableBranch

有时候希望实现类似 ifelse 的条件分支，使用 RunnableBrach 可以接受 (条件函数, runnable) 的元组对和一个默认分支。

In [225]:
from langchain_core.runnables import RunnableBranch, chain

@chain
def myfunc(x):
    return(x * 2)

branch = RunnableBranch(
    (lambda x: isinstance(x, str), lambda x: x.upper()),
    (lambda x: isinstance(x, int), myfunc),
    lambda x: "goodbye",
)
branch.get_graph().print_ascii()

print(branch.invoke("hello")) # "HELLO"
print(branch.invoke(3)) # 6
print(branch.invoke(None)) # "goodbye"

+-------------+  
| BranchInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Branch |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| BranchOutput | 
+--------------+ 
HELLO
6
goodbye


### RouterRunnable

条件分支中还有一种情况更为简单，适合使用RouterRunnable，可以比RunnableBranch简洁得多。
<br>但传递参数时必须以 RouterInput 类型传入，或使用等价的字典结构。

```
class RouterInput(TypedDict):
    key: str
    input: Any
```

In [226]:
from langchain_core.runnables import RouterRunnable, chain

@chain
def lower(x):
    return(x.lower())

@chain
def upper(x):
    return(x.upper())

router = RouterRunnable({"lower": lower, "upper": upper})
router.get_graph().print_ascii()

print(router.invoke({"key": "upper", "input": "Hello"}))
print(router.invoke({"key": "lower", "input": "Hello"}))

+---------------------+  
| RouterRunnableInput |  
+---------------------+  
            *            
            *            
            *            
   +----------------+    
   | RouterRunnable |    
   +----------------+    
            *            
            *            
            *            
+----------------------+ 
| RouterRunnableOutput | 
+----------------------+ 
HELLO
hello


## 大模型

### 大模型相关类结构

- BaseLanguageModel（大模型）
    - BaseChatModel `[LanguageModelInput, LanguageModelOutputVar]`
        - SimpleChatModel
    - BaseLLM
        - HuggingFacePipeline
        - LLM
            - FakeListLLM
                - FakeStreamingListLLM
            - HumanInputLLM
            - HuggingFaceHub
            - HuggingFaceEndpoint
            - HuggingFaceTextGenInference
            - QianfanLLMEndpoint
            - BaichuanLLM
        - BaseOpenAI
            - OpenAI
        - OpenAIChat
        - Tongyi
        - Ollama

### 定制大模型

## 提示语

### 提示语相关类结构

- BasePromptTemplate `[Dict, PromptValue]`
    - StringPromptTemplate
        - PromptTemplate
        - FewShotPromptWithTemplates
        - FewShotPromptTemplate
    - BaseChatPromptTemplate
        - ChatPromptTemplate
    - ImagePromptTemplate
    - PipelinePromptTemplate

### 提示语模板库

### 从文件加载提示语

## 检索器

### 相关类结构

- BaseRetriever `[RetrieverInput, RetrieverOutput]`
    - VectorStoreRetriever
    - AzureCognitiveSearchRetriever
    - AmazonKnowledgeBasesRetriever
    - ChatGPTPluginRetriever
    - ElasticSearchBM25Retriever
    - KNNRetriever
    - LlamaIndexRetriever
    - MetalRetriever
    - MilvusRetriever
    - OutlineRetriever
    - PineconeHybridSearchRetriever
    - QdrantSparseVectorRetriever
    - RemoteLangChainRetriever
    - SVMRetriever
    - TavilySearchAPIRetriever
    - TFIDFRetriever
    - WeaviateHybridSearchRetriever
    - WikipediaRetriever


## Tool

### Tool相关类结构

- BaseTool `[Union[str, Dict], Any]`
    - Tool
    - StructuredTool
    - HumanInputRun
    - GitHubAction
    - GitLabAction
    - JsonListKeysTool
    - JsonGetValueTool
    - Memorize
    - OpenWeatherMapQueryRun
    - SleepTool
    - ShellTool
    - QuerySQLDataBaseTool
    - InfoSQLDatabaseTool
    - QuerySQLCheckerTool
    - TavilySearchResults
    - TavilyAnswer
    - VectorStoreQATool
    - VectorStoreQAWithSourcesTool
    - WikipediaQueryRun

    **Files:**
    - CopyFileTool
    - DeleteFileTool
    - FileSearchTool
    - ListDirectoryTool
    - MoveFileTool
    - ReadFileTool
    - WriteFileTool


### 定制工具

## 输出解析

### 相关类结构

- BaseGenerationOutputParser `[Union[str, BaseMessage], T]`
    - OutputFunctionsParser
        - PydanticOutputFunctionsParser
            - PydanticAttrOutputFunctionsParser
    - JsonOutputToolsParser
        - JsonOutputKeyToolsParser
        - PydanticToolsParser
- BaseOutputParser
    - BaseTransformOutputParser
        - BaseCumulativeTransformOutputParser
            - JsonOutputParser（别名SimpleJsonOutputParser）
            - JsonOutputFunctionsParser
                - JsonKeyOutputFunctionsParser
        - StrOutputParser
        - XMLOutputParser
        - ListOutputParser
            - CommaSeparatedListOutputParser
            - NumberedListOutputParser
            - MarkdownListOutputParser
    - BooleanOutputParser
    - CombiningOutputParser
    - DatetimeOutputParser
    - EnumOutputParser
    - OutputFixingParser（使用LLM修复错误）
    - PandasDataFrameOutputParser
    - PydanticOutputParser
    - RegexDictParser
    - RegexParser
    - RetryOutputParser
    - RetryWithErrorOutputParser
    - StructuredOutputParser
    - YamlOutputParser


### 定制一个输出解析器

## 已实现Chains     

- Chain
    - LLMChain
        - ConversationChain
        - QuestionGeneratorChain
        - FlareChain
    - LLMCheckerChain
    - LLMRequestsChain
    - LLMMathChain（使用python代码执行数学计算）
    - LLMSummarizationCheckerChain
    - MapReduceChain
    - OpenAIModerationChain
    - SequentialChain
    - SimpleSequentialChain
    - APIChain
    - BaseCombineDocumentsChain
    - AnalyzeDocumentChain
    - ConstitutionalChain
    - BaseConversationalRetrievalChain
        - ConversationalRetrievalChain
        - ChatVectorDBChain
    - ElasticsearchDatabaseChain
    - NatBotChain（实现一个基于LLM的浏览器）
    - QAGenerationChain（问答对生成）
    - BaseQAWithSourcesChain
        - QAWithSourcesChain
    - BaseRetrievalQA
        - RetrievalQA
        - VectorDBQA
    - RouterChain
        - MultiRouteChain
            - MultiRetrievalQAChain
        - EmbeddingRouterChain
        - LLMRouterChain

## 输入输出类型

### 消息

### 文档

## 记忆

## RAG

### 文本处理

### 嵌入模型

### 向量库

### 检索优化

## 智能体

### 代码结构

### 工具箱

### 定制智能体

## 支持的LLM

## 支持的向量数据库

## 项目模板